# Exploratory Data Analysis #1

### Import libraries

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.recommendation import ALS

from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pyspark
spark = SparkSession.builder.getOrCreate()

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.types import *

In [2]:
import os
import pandas as pd

### Import data

In [54]:
df2 = pd.read_csv('comb_df7.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54387 entries, 0 to 54386
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genre             54387 non-null  object 
 1   title             54387 non-null  object 
 2   author            54387 non-null  object 
 3   pages             54387 non-null  int64  
 4   average_rating    54387 non-null  float64
 5   book_description  40773 non-null  object 
 6   user_unique_key   54387 non-null  int64  
 7   number_of_stars   54387 non-null  int64  
 8   book_unique_key   54387 non-null  int64  
 9   review_text       53340 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 4.1+ MB


### Inspect the data

### Pre-process the data

In [ ]:
#join the tables

In [ ]:
#check for nulls/nans

### EDA visualizations

### Build first simple model (FSM)

In [64]:
dfpd= pd.read_csv('comb_df7.csv')

In [68]:

schema = StructType([StructField("genre", StringType(), True),StructField("title", StringType(), True), 
                     StructField("author", StringType(), True), StructField("pages", IntegerType(), True), 
                     StructField("average_rating", FloatType(), True), StructField("book_description", StringType(), True), 
                     StructField("user_unique_key", IntegerType(), True),StructField("number_of_stars", IntegerType(), True), 
                     StructField("book_unique_key", IntegerType(), True), StructField("review_text", StringType(), True)])

sp_df = spark.createDataFrame(dfpd, schema=schema)

In [69]:
sp_df.dtypes

[('genre', 'string'),
 ('title', 'string'),
 ('author', 'string'),
 ('pages', 'int'),
 ('average_rating', 'float'),
 ('book_description', 'string'),
 ('user_unique_key', 'int'),
 ('number_of_stars', 'int'),
 ('book_unique_key', 'int'),
 ('review_text', 'string')]

In [3]:
sm_df= pd.read_csv('small_df.csv')

In [4]:
schema = StructType([StructField("genre", StringType(), True),StructField("title", StringType(), True), 
                     StructField("author", StringType(), True), StructField("pages", IntegerType(), True), 
                     StructField("average_rating", FloatType(), True), StructField("book_description", StringType(), True), 
                     StructField("user_unique_key", IntegerType(), True),StructField("number_of_stars", IntegerType(), True), 
                     StructField("book_unique_key", IntegerType(), True), StructField("review_text", StringType(), True)])

df_sm2 = spark.createDataFrame(sm_df, schema=schema)

In [ ]:
# pddf = pd.read_csv("comb_df7.csv")
# sp_df = spark.createDataFrame(pd_df, schema=schema)

In [92]:

print("The book with the fewest ratings: ")
out3.groupBy("book_unique_key").count().select(min("count")).show()

# print("Avg num ratings per book: ")
# out3.groupBy("book_unique_key").count().select(avg("count")).show()


# print("User with the fewest ratings: ")
# out3.groupBy("user_unique_key").count().select(min("count")).show()

# print("Avg num ratings per user: ")
# out3.groupBy("user_unique_key").count().select(avg("count")).show()

The book with the fewest ratings: 


AnalysisException: cannot resolve '`c`' given input columns: [book_unique_key, count];;
'Project ['c]
+- Aggregate [book_unique_key#1780], [book_unique_key#1780, count(1) AS count#3566L]
   +- Project [genre#1749, title#1750, author#1751, pages#1752, average_rating#1753, book_description#1754, cast(user_unique_key#1755 as int) AS user_unique_key#1791, number_of_stars#1769, book_unique_key#1780, review_text#1758]
      +- Project [genre#1749, title#1750, author#1751, pages#1752, average_rating#1753, book_description#1754, user_unique_key#1755, number_of_stars#1769, cast(book_unique_key#1757 as int) AS book_unique_key#1780, review_text#1758]
         +- Project [genre#1749, title#1750, author#1751, pages#1752, average_rating#1753, book_description#1754, user_unique_key#1755, cast(number_of_stars#1756 as int) AS number_of_stars#1769, book_unique_key#1757, review_text#1758]
            +- Relation[genre#1749,title#1750,author#1751,pages#1752,average_rating#1753,book_description#1754,user_u

"sparsity is calculated by the number of cells in a matrix that contain a rating divided by the total number of values that matrix could hold given the number of users and items [such as books]. In other words, dividing the number of ratings present in the matrix by the product of users and [books] in the matrix and subtracting that from 1 will give us the sparsity or the percentage of the ratings matrix that is empty."

In [74]:
# calculating sparcity--

numerator = out3.select("number_of_stars").count()


num_users = out3.select("user_unique_key").distinct().count()
num_books = out3.select("book_unique_key").distinct().count()


denominator = num_users * num_books


sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.25% empty.


In [5]:
numerator = df_sm2.select("number_of_stars").count()


num_users = df_sm2.select("user_unique_key").distinct().count()
num_books = df_sm2.select("book_unique_key").distinct().count()


denominator = num_users * num_books


sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  99.60% empty.


In [6]:
train, test = sp_df.randomSplit([0.8, 0.2])

NameError: name 'sp_df' is not defined

In [71]:
als = ALS(maxIter=5,rank=4, 
          userCol='user_unique_key', itemCol='book_unique_key', 
          ratingCol='number_of_stars',
          coldStartStrategy='drop')


model_fsm = als.fit(train)

In [73]:
predictions = model_fsm.transform(test)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='number_of_stars',
                                predictionCol='prediction')
rmse = evaluator.evaluate(predictions)
print('Root-mean-square error = ' + str(rmse))

Root-mean-square error = 2.0271723460408415


In [87]:

evaluator = RegressionEvaluator(metricName='mae', labelCol='number_of_stars',
                                predictionCol='prediction')
mae = evaluator.evaluate(predictions)
print('mean absolute error = ' + str(mae))

mean absolute error = 1.4594755863126267


In [7]:
train, test = df_sm2.randomSplit([0.8, 0.2])

In [9]:
#train, test = df_sm2.randomSplit([0.8, 0.2])
als2 = ALS(maxIter=5,rank=4, 
          userCol='user_unique_key', itemCol='book_unique_key', 
          ratingCol='number_of_stars',
          coldStartStrategy='drop')


model_fsm2 = als2.fit(train)
predictions2 = model_fsm2.transform(test)
predictions2.show()

+----------------+--------------------+-----------+-----+--------------+--------------------+---------------+---------------+---------------+--------------------+-----------+
|           genre|               title|     author|pages|average_rating|    book_description|user_unique_key|number_of_stars|book_unique_key|         review_text| prediction|
+----------------+--------------------+-----------+-----+--------------+--------------------+---------------+---------------+---------------+--------------------+-----------+
|biography-memoir|This Is Going to ...|Adam Kay ()|  256|          4.41|Welcome to the li...|            255|              4|            148|this is an overal...|   4.306255|
|biography-memoir|This Is Going to ...|Adam Kay ()|  256|          4.41|Welcome to the li...|           2542|              4|            148|Adam Kay had come...|   3.719646|
|biography-memoir|This Is Going to ...|Adam Kay ()|  256|          4.41|Welcome to the li...|           3538|              4|

In [98]:
predictions2 = model_fsm2.transform(test)
evaluator2 = RegressionEvaluator(metricName='rmse', labelCol='number_of_stars',
                                predictionCol='prediction')
rmse2 = evaluator2.evaluate(predictions2)
print('Root-mean-square error = ' + str(rmse2))

Root-mean-square error = 2.0971997522129304


In [99]:
evaluator3 = RegressionEvaluator(metricName='mae', labelCol='number_of_stars',
                                predictionCol='prediction')
mae2 = evaluator3.evaluate(predictions2)
print('mean absolute error = ' + str(mae2))

mean absolute error = 1.4678074366732219


In [11]:
train, test = df_sm2.randomSplit([0.8, 0.2])

In [12]:
als_model = ALS(userCol='user_unique_key', itemCol='book_unique_key',ratingCol='number_of_stars', 
                rank=10  , maxIter=20 , regParam=.05,
                nonnegative=True,implicitPrefs=False, 
                coldStartStrategy='drop')
model4 = als_model.fit(train)
pred = model4.transform(test)
pred.show()

+----------------+--------------------+--------------------+-----+--------------+--------------------+---------------+---------------+---------------+--------------------+----------+
|           genre|               title|              author|pages|average_rating|    book_description|user_unique_key|number_of_stars|book_unique_key|         review_text|prediction|
+----------------+--------------------+--------------------+-----+--------------+--------------------+---------------+---------------+---------------+--------------------+----------+
|biography-memoir|This Is Going to ...|         Adam Kay ()|  256|          4.41|Welcome to the li...|          24890|              3|            148|Interesting insig...| 4.3648696|
|biography-memoir|This Is Going to ...|         Adam Kay ()|  256|          4.41|Welcome to the li...|            255|              4|            148|this is an overal...|  4.057796|
|biography-memoir|This Is Going to ...|         Adam Kay ()|  256|          4.41|Welc

In [14]:

evaluator4 = RegressionEvaluator(metricName='rmse', labelCol='number_of_stars',
                                predictionCol='prediction')
rmse4 = evaluator4.evaluate(pred)
print('Root-mean-square error = ' + str(rmse4))

Root-mean-square error = 1.3042900375870623


In [10]:
train, test = df_sm2.randomSplit([0.8, 0.2])

In [30]:
train, test = df_sm2.randomSplit([0.8, 0.2])
als_modelv = ALS(userCol='user_unique_key', itemCol='book_unique_key',ratingCol='number_of_stars',
                nonnegative=True,implicitPrefs=False, 
                coldStartStrategy='drop')

# modelv = als_modelv.fit(train)
# pred = modelv.transform(test)
                
evaluatorv = RegressionEvaluator(metricName='rmse', labelCol='number_of_stars',
                                predictionCol='prediction')
params=ParamGridBuilder()
params = ParamGridBuilder().addGrid(als_modelv.regParam, [0.01, 0.05, .1, 1.5]).build()
                #.addGrid(als_modelv.rank, [4, 10, 30, 70])\
                #.addGrid(als_modelv.maxIter[5, 15, 30, 45, 75])\
                #.build()
    
# cv= CrossValidator()
crossval = CrossValidator(estimator=als_modelv, 
                          estimatorParamMaps=params,
                          evaluator=evaluatorv,
                          numFolds= 6)
modelv= crossval.fit(train)    

best_model = modelv.bestModel

# best_model.bestModel.rank

Does it work? do the recommendations make sense?

"when ALS generates recommendations, they are provided in descending order by userId, so there's no need to sort the dataframe."

In [ ]:
origin_rating =
recommended=
print("User #25 Ratings:")
origin_ratings.filter(col("user_unique_key") == 60).sort("number_of_stars", ascending= False).show()


print("User 300 Recommendations:")
recommended.filter(col("user_unique_key") == 300).show()

